In [1]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'caret' was built under R version 3.6.3"
Loading required package: lattice

Warning message:
"package 'lattice' was built under R version 3.6.2"
Loading required package: ggplot2

Warning message:
"package 'tidyverse' was built under R version 3.6.3"
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v tibble  3.1.0     v purrr   0.3.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

Warning message:
"package 'tibble' was built under R version 3.6.2"
Warning message:
"package 'tidyr' was built under R version 3.6.3"
Warning message:
"package 'readr' was built under R version 3.6.3"
Warning message:
"pack

In [2]:
dim(df_finale)

[1] 1403   74

In [3]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [4]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [5]:
dim(test)
dim(train)

[1] 421 175

[1] 982 175

In [6]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    491     491 

## Model

In [7]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [8]:

model_svmPoly <- train(x=TrainData, y=TrainClasses, 
               method = "svmPoly", scale= FALSE) 

In [9]:

model_svmRadial <- train(x=TrainData, y=TrainClasses, 
               method = "svmRadial", scale= FALSE) 

Warning message in .local(x, ...):
"Variable(s) `' constant. Cannot scale data."


In [10]:

model_svmLinear <- train(x=TrainData, y=TrainClasses, 
               method = "svmLinear", scale= FALSE)

# Results

In [11]:
print(model_svmPoly)
plot(model_svmPoly,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Polynomial Kernel 

982 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 982, 982, 982, 982, 982, 982, ... 
Resampling results across tuning parameters:

  degree  scale  C     Accuracy   Kappa    
  1       0.001  0.25  0.6218879  0.2600317
  1       0.001  0.50  0.6957583  0.3965601
  1       0.001  1.00  0.7191033  0.4404637
  1       0.010  0.25  0.7296920  0.4605310
  1       0.010  0.50  0.7332805  0.4675078
  1       0.010  1.00  0.7337522  0.4683320
  1       0.100  0.25  0.7336689  0.4678741
  1       0.100  0.50  0.7309025  0.4621859
  1       0.100  1.00  0.7226712  0.4458390
  2       0.001  0.25  0.6980876  0.4009434
  2       0.001  0.50  0.7212304  0.4446130
  2       0.001  1.00  0.7294688  0.4601964
  2       0.010  0.25  0.7372257  0.4753688
  2       0.010  0.50  0.7372642  0.4752046
  2       0.010  1.00  0.7399242  0.4805103
  2       0.100  0.25  0.74

In [12]:
print(model_svmRadial)
plot(model_svmRadial,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Radial Basis Function Kernel 

982 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 982, 982, 982, 982, 982, 982, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.7388155  0.4774927
  0.50  0.7443502  0.4887326
  1.00  0.7517268  0.5033451

Tuning parameter 'sigma' was held constant at a value of 0.01828082
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.01828082 and C = 1.


In [13]:
print(model_svmLinear)
plot(model_svmLinear)

Support Vector Machines with Linear Kernel 

982 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 982, 982, 982, 982, 982, 982, ... 
Resampling results:

  Accuracy   Kappa    
  0.7087853  0.4172891

Tuning parameter 'C' was held constant at a value of 1


ERROR: Error in plot.train(model_svmLinear): There are no tuning parameters with more than 1 value.


In [14]:
# collect resamples
results <- resamples(list(Poly=model_svmPoly,  Radial=model_svmRadial, Linear=model_svmLinear))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: Poly, Radial, Linear 
Number of resamples: 25 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.7197802 0.7382920 0.7449857 0.7471916 0.7609890 0.7733711    0
Radial 0.7217631 0.7403315 0.7549858 0.7517268 0.7596899 0.7765363    0
Linear 0.6648352 0.6946779 0.7107438 0.7087853 0.7250000 0.7547170    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.4402009 0.4755337 0.4942665 0.4950572 0.5205305 0.5468695    0
Radial 0.4436486 0.4809957 0.5098402 0.5033451 0.5193959 0.5522341    0
Linear 0.3299940 0.3893030 0.4212636 0.4172891 0.4488092 0.5121454    0


# Prediction

In [15]:
pred_svmPoly <- predict(model_svmPoly,TestData)
pred_svmPoly.prob <- predict(model_svmPoly,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmPoly) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      91      56
   NotFast   85     189
                                          
               Accuracy : 0.6651          
                 95% CI : (0.6178, 0.7101)
    No Information Rate : 0.5819          
    P-Value [Acc > NIR] : 0.0002844       
                                          
                  Kappa : 0.2953          
                                          
 Mcnemar's Test P-Value : 0.0183724       
                                          
            Sensitivity : 0.5170          
            Specificity : 0.7714          
         Pos Pred Value : 0.6190          
         Neg Pred Value : 0.6898          
             Prevalence : 0.4181          
         Detection Rate : 0.2162          
   Detection Prevalence : 0.3492          
      Balanced Accuracy : 0.6442          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5170455            0.7714286            0.6190476 
      Neg Pred Value            Precision               Recall 
           0.6897810            0.6190476            0.5170455 
                  F1           Prevalence       Detection Rate 
           0.5634675            0.4180523            0.2161520 
Detection Prevalence    Balanced Accuracy 
           0.3491686            0.6442370

In [16]:
pred_svmRadial <- predict(model_svmRadial,TestData)
pred_svmRadial.prob <- predict(model_svmRadial,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmRadial) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     100      47
   NotFast   61     213
                                          
               Accuracy : 0.7435          
                 95% CI : (0.6989, 0.7845)
    No Information Rate : 0.6176          
    P-Value [Acc > NIR] : 3.171e-08       
                                          
                  Kappa : 0.4478          
                                          
 Mcnemar's Test P-Value : 0.211           
                                          
            Sensitivity : 0.6211          
            Specificity : 0.8192          
         Pos Pred Value : 0.6803          
         Neg Pred Value : 0.7774          
             Prevalence : 0.3824          
         Detection Rate : 0.2375          
   Detection Prevalence : 0.3492          
      Balanced Accuracy : 0.7202          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.6211180            0.8192308            0.6802721 
      Neg Pred Value            Precision               Recall 
           0.7773723            0.6802721            0.6211180 
                  F1           Prevalence       Detection Rate 
           0.6493506            0.3824228            0.2375297 
Detection Prevalence    Balanced Accuracy 
           0.3491686            0.7201744

In [17]:
pred_svmLinear <- predict(model_svmLinear,TestData)
pred_svmLinear.prob <- predict(model_svmLinear,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmLinear) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      99      48
   NotFast   87     187
                                          
               Accuracy : 0.6793          
                 95% CI : (0.6324, 0.7237)
    No Information Rate : 0.5582          
    P-Value [Acc > NIR] : 2.488e-07       
                                          
                  Kappa : 0.3353          
                                          
 Mcnemar's Test P-Value : 0.001074        
                                          
            Sensitivity : 0.5323          
            Specificity : 0.7957          
         Pos Pred Value : 0.6735          
         Neg Pred Value : 0.6825          
             Prevalence : 0.4418          
         Detection Rate : 0.2352          
   Detection Prevalence : 0.3492          
      Balanced Accuracy : 0.6640          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5322581            0.7957447            0.6734694 
      Neg Pred Value            Precision               Recall 
           0.6824818            0.6734694            0.5322581 
                  F1           Prevalence       Detection Rate 
           0.5945946            0.4418052            0.2351544 
Detection Prevalence    Balanced Accuracy 
           0.3491686            0.6640014

In [18]:
roc_obj <- roc(TestClasses, as.numeric(pred_svmRadial))
plot(roc_obj, main="Curva ROC")


Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [19]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_svmRadial))

Data: as.numeric(pred_svmRadial) in 147 controls (TestClasses fast) < 274 cases (TestClasses NotFast).
Area under the curve: 0.7288

In [20]:
pred_svmRadial.prob

fast,NotFast
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
